<a href="https://colab.research.google.com/github/omkar806/Assignments/blob/main/Swipe_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pypdf openai tiktoken pdf2image
!pip install google-generativeai
!apt-get install -y poppler-utils

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.7/360.7 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.1 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 45 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.5 [186 kB]
Fetched 186 

In [2]:
from pdf2image import convert_from_path
import google.generativeai as genai
from google.colab import userdata
import os
import imghdr
import PIL.Image

In [4]:
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
# GOOGLE_API_KEY = input("Please enter your Gemini API Key from Google AI studio : ")
genai.configure(api_key=GOOGLE_API_KEY)

In [5]:
model = genai.GenerativeModel('gemini-1.5-flash')

In [6]:
def system_prompt()->str:
    system_prompt = f"""You are a Invoice/Receipt Analysing tool . Anaylse the image of the invoice provided and extract information from the following receipt image and return a JSON object with these exact keys:Customer_Details,Products,Total Amount.\n
    Rules:
    1. For total_cost, use the highest monetary value in the text.
    2. Customer_Details will consist of another json object consisting the keys Customer_Name,Customer_Address,Email Address,Phone Number,Customer ID,Billing Address,Shipping Address,Account Number,Tax ID/VAT Number,Company Name,Payment Method.
    3.Products will consist of another json object consisting of keys Product_name_1,quantity,unit_price.
    6. If any value is not found, return null.
    7. If all values are null, return null.
    Ensure the strictly that output is a valid JSON object containing strictly the above keys, without any explanations.
    Generate a JSON response in the following format without using the ```json block. Ensure the output is properly formatted as plain text JSON.\n
    \n
    """
    return system_prompt

In [7]:
#Function to generate a json with required keys
def get_invoice_details(image):
  response = model.generate_content([system_prompt(), image], stream=True)
  response.resolve()
  print(response.text)


In [8]:
def save_pdf_as_image(images):
  imgs=[]
  for i, image in enumerate(images):
    image.save(f'page_{i+1}.png', 'PNG')
    imgs.append(f'/content/page_{i+1}.png')
  return imgs

In [9]:
files_list = ['/content/Sample Invoice image.png','/content/Sample Invoice.pdf','/content/c669abb4-f485-4880-8973-cc7fdfeee22e.pdf']
images = []
# Iterate over the files and process them
for file in files_list:
  print(file)
  if imghdr.what(file):
    img = PIL.Image.open(file)
    print("pinting image")
    print(img)
    images.append(img)
  elif file.lower().endswith('.pdf'):
    img = convert_from_path(file)
    for imgg in img:
      images.append(imgg)
  else:
    print(f"{file} is neither an image nor a PDF.")

for img in images:
  print("Initiating the task...")
  get_invoice_details(img)
  print("\n")







/content/Sample Invoice image.png
pinting image
<PIL.PngImagePlugin.PngImageFile image mode=RGBA size=907x962 at 0x7D8A937D1210>
/content/Sample Invoice.pdf
/content/c669abb4-f485-4880-8973-cc7fdfeee22e.pdf
Initiating the task...
{
"Customer_Details": {
"Customer_Name": "Test",
"Customer_Address": "Hyderabad, TELANGANA, 500089",
"Email Address": "test@gmail.com",
"Phone Number": "9108239284",
"Customer ID": null,
"Billing Address": null,
"Shipping Address": null,
"Account Number": "1234567890",
"Tax ID/VAT Number": null,
"Company Name": null,
"Payment Method": null
},
"Products": {
"Product_name_1": "WASTE AND SCRAP OF STAINLESS STEEL",
"quantity": "6,790 KGS",
"unit_price": "95.00"
},
"Total Amount": "7,68,771.00"
}


Initiating the task...
{
"Customer_Details": {
"Customer_Name": "TEST",
"Customer_Address": "Test\nHyderabad, TELANGANA, 500089",
"Email Address": "test@gmail.com",
"Phone Number": "9108239284",
"Customer ID": null,
"Billing Address": null,
"Shipping Address": "Test\nHyd